In [1]:
import cv2
import dlib
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
os.chdir(parent_dir)
os.getcwd()

'/Users/pareshar/Personal/Github/temp/Facial-feature-detector'

In [3]:
# static variables
path_to_images = "data/images_age_gender/"
image_files = os.listdir(path_to_images)
face_detector_weights = "models/face_detection/res10_300x300_ssd_iter_140000.caffemodel"
face_detector_config = "models/face_detection/deploy.prototxt.txt"
age_weights = "models/face_age/age_net.caffemodel"
age_config = "models/face_age/age_deploy.prototxt"
gender_weights = "models/face_gender/gender_net.caffemodel"
gender_config = "models/face_gender/gender_deploy.prototxt"
age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
gender_list = ['Male', 'Female']
model_mean = (78.4263377603, 87.7689143744, 114.895847746) # taken from paper

In [4]:
df = pd.DataFrame(columns=["file_name", "model", "confidence_face_detected", "age_range", "age_confidence", "gender", "gender_confidence"])
df_list = []


for image_file in image_files:
    image = cv2.imread(path_to_images + image_file)
    h, w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image=image, scalefactor=1.0, size=(300, 300))
    
    face_detector_net = cv2.dnn.readNetFromCaffe(face_detector_config, face_detector_weights)
    face_detector_net.setInput(blob)
    face_detections = face_detector_net.forward()  
    age_net = cv2.dnn.readNet(age_weights, age_config)
    gender_net = cv2.dnn.readNet(gender_weights, gender_config)
    
    d = None
    
    for i in range(0, face_detections.shape[2]):
        confidence = face_detections[0, 0, i, 2]
        if confidence > 0.97:
            box = face_detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            face = image[startY:endY, startX:endX]
            
            blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), model_mean, swapRB=False)
            
            age_net.setInput(blob)
            age_preds = age_net.forward()
            i = age_preds[0].argmax()
            age = age_list[i]
            age_confidence_score = age_preds[0][i]
            
            gender_net.setInput(blob)
            gender_preds = gender_net.forward()
            i = gender_preds[0].argmax()
            gender = gender_list[i]
            gender_confidence_score = gender_preds[0][i]
            
            # plt.imshow(face)
            # plt.show()  
            
            d = {
            "file_name": image_file,
            "model": "ageNet",
            "confidence_face_detected": confidence,
            "age_range": age,
            "age_confidence": age_confidence_score,
            "gender": gender,
            "gender_confidence": gender_confidence_score 
            }
            df_list.append(d)
            break
        
    if d is None or image_file != d["file_name"]:
            
        d = {
            "file_name": image_file,
            "model": "ageNet",
            "confidence_face_detected": confidence,
            "age_range": "NA",
            "age_confidence": "NA",
            "gender": "NA",
            "gender_confidence": "NA" 
            }
                
        df_list.append(d)
    
df = pd.concat([df, pd.DataFrame(df_list)], ignore_index=True).round(2)

In [5]:
df.sort_values("file_name")

,file_name,model,confidence_face_detected,age_range,age_confidence,gender,gender_confidence
1,22_me.jpg,ageNet,0.98,(25-32),0.67165,Male,1.0
3,25_32_woman.jpg,ageNet,1.00,(25-32),0.859894,Female,0.952863
2,38_43_man.jpg,ageNet,1.00,(25-32),0.681306,Male,0.999431
8,38_43_woman.jpg,ageNet,0.99,(48-53),0.886763,Female,0.998737
7,4_6_boy.jpg,ageNet,0.99,(4-6),0.639939,Male,0.999049
4,4_6_girl.jpg,ageNet,0.99,(4-6),0.319971,Female,0.998801
6,60_100_man.jpg,ageNet,0.99,(4-6),0.548595,Male,0.999973
5,60_100_woman.jpg,ageNet,1.00,(60-100),0.332936,Female,0.984078
9,60_100_woman_2.jpg,ageNet,1.00,(38-43),0.414388,Male,0.518144
0,minion.jpg,ageNet,0.00,NA,NA,NA,NA


# Hugging face pre-trained VIT model

In [5]:
# age

import os
import cv2
from transformers import ViTImageProcessor, ViTForImageClassification

os.environ[
        "CURL_CA_BUNDLE"
    ] = ""  # fixes VPN issue when connecting to hugging face hub


image = cv2.imread("data/4_6_boy.jpg")


# Init model, transforms
model = ViTForImageClassification.from_pretrained('nateraw/vit-age-classifier')
transforms = ViTImageProcessor.from_pretrained('nateraw/vit-age-classifier')

# Transform our image and pass it through the model
inputs = transforms(image, return_tensors='pt')
output = model(**inputs)

# Predicted Class probabilities
proba = output.logits.softmax(1)

# Predicted Classes
preds = proba.argmax(1)


/Users/pareshar/.pyenv/versions/3.8.10/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/Users/pareshar/.pyenv/versions/3.8.10/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pareshar/.pyenv/versions/3.8.10/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/p

In [24]:
max(proba[0]).item()

0.7176125645637512

In [7]:
id2label = {
    0: "0-2",
    1: "3-9",
    2: "10-19",
    3: "20-29",
    4: "30-39",
    5: "40-49",
    6: "50-59",
    7: "60-69",
    8: "more than 70"
  }

id2label[int(preds)]

'3-9'

In [28]:
# gender

import os
import cv2
from transformers import ViTImageProcessor, ViTForImageClassification

os.environ[
        "CURL_CA_BUNDLE"
    ] = ""  # fixes VPN issue when connecting to hugging face hub


image = cv2.imread("data/gigi_hadid.webp")


# Init model, transforms
model = ViTForImageClassification.from_pretrained('rizvandwiki/gender-classification')
transforms = ViTImageProcessor.from_pretrained('rizvandwiki/gender-classification')

# Transform our image and pass it through the model
inputs = transforms(image, return_tensors='pt')
output = model(**inputs)

# Predicted Class probabilities
proba = output.logits.softmax(1)

# Predicted Classes
preds = proba.argmax(1)


/Users/pareshar/.pyenv/versions/3.8.10/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/pareshar/.pyenv/versions/3.8.10/lib/python3.8/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [29]:
max(proba[0]).item()

0.9677436351776123

In [30]:
id2label = {
    0: "female",
    1: "male",
  }

id2label[int(preds)]

'female'

# Other
- Dataset used to train model: https://talhassner.github.io/home/projects/Adience/Adience-data.html#agegender